# Using high-order Lagrange elements

NekRS uses spectral element method, with Lagrange basis functions used on Gauss-Labatto-Legendre quadrature points. The exhibits exponential convergence under $p$-refinement. By default, VTK splits each spectral elements $(n+1)^3$ linear elements. Fairly recent additions to VTK mean that it can handle high-order Lagrange elements and means we have the potential to retain spectral accuracy in post-processing. 

In this tutorial, we convert the `UnstructuredGrid` from NekRS and convert it to use `vtkLagrangeHexahedron` elements. Currrently, it is not entirely clear if/when such a conversion will be useful, but it may allow for a more accurate of derivatives and maybe [vortex identification](../visualising_vortices.ipynb). 

The guts of this conversion is found in [`lagrange_convert.py`](lagrange_convert.py). It is fairly opaque but the core idea is that each spectral element must be extracted from original data and then the data must be re-ordered as VTK vertices must be order in a specific way. More information on high-order elements can be found [here](https://www.kitware.com/modeling-arbitrary-order-lagrange-finite-elements-in-the-visualization-toolkit/). The method used below is slow and would be much faster if implemented into the reader itself. Due to its slow speed, we will do this on the pebble case (the turbulent pipe flow takes up to 9 minutes).

## Reading the data

First, we read the data and use the `enable_spectral_element_ids` options, as this is how the each spectral element can be extracted and converted.

In [16]:
import pyvista as pv
from pathlib import Path

path_base = Path("../pebble_cht")

# get_reader function uses the file extensions to determine which reader should be used
fluid_reader = pv.get_reader(path_base / "pebble.nek5000")
fluid_reader.enable_spectral_element_ids()
fluid = fluid_reader.read()

print(fluid.cell_data)

3D-Mesh found, spectral element of size = 4pyvista DataSetAttributes
Association     : CELL
Active Scalars  : None
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays :
    spectral element id     uint32     (51408,)
*4*4=64


### Converting the mesh to use Lagrange Hexahedron

In [17]:
from lagrange_convert import to_lagrange_hexahedron

new_grid = to_lagrange_hexahedron(fluid)

## View and compare with original

For such a simple flow configuration, we do not expect any signficant difference between the two.

In [18]:
p = pv.Plotter(shape=(1,2), window_size=(250,600), border=False)
p.subplot(0,0)

p.add_mesh(fluid.clip(normal='x'),
           scalars='Temperature',
           cmap='bwr',
           show_scalar_bar=False)

p.subplot(0,1)

p.add_mesh(new_grid.clip(normal='x'),
           scalars='Temperature',
           cmap='bwr',
           show_scalar_bar=False)

p.link_views()
p.camera.zoom(2.)
p.show()


Widget(value='<iframe src="http://localhost:35149/index.html?ui=P_0x77614455ee30_13&reconnect=auto" class="pyv…